#### 统计 test 数据集中的 注释词性特征
平均长度为 10.15 ， 查看了 长度为 10 和 11 的注释词性特征情况 

In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag


In [2]:
def retrieve_test_data():
    test_param = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/param/test.json")
    test_return = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/return/test.json")
    test_summary = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/summary/test.json")
    test_df = pd.concat([test_summary,test_param, test_return], axis=0)
    test_df = test_df.reset_index(drop=True)
    return test_df
test_df = retrieve_test_data()

In [3]:
test_df.loc[0]['old_comment_raw']

'Creates elastic node as single member of a cluster.'

In [4]:
test_df.loc[1]['old_comment_raw']

'Check whether any insertion or deletion actions are currently queued.'

In [5]:
comment_sentence = test_df['old_comment_raw']

In [6]:
def get_WordTagging_attribution_for_sentence(comment_sentence_list):
    tagged_sentence_list = []
    for sentence in comment_sentence_list:
        words = word_tokenize(sentence)
        tagged_words = pos_tag(words)
        tagged_sentence_list.append(tagged_words)
        
    return tagged_sentence_list
tagged_sentence_list = get_WordTagging_attribution_for_sentence(comment_sentence)

In [7]:
tagged_sentence_list[0]

[('Creates', 'NNS'),
 ('elastic', 'JJ'),
 ('node', 'NN'),
 ('as', 'IN'),
 ('single', 'JJ'),
 ('member', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('cluster', 'NN'),
 ('.', '.')]

In [8]:
from collections import Counter

def static_tagging_type(data):
    punctuations = {'.', ',', ';', ':', '(', ')', '[', ']', '{', '}', '!', '?', '``', "''","#"}

    second_elements = [tag for sentence in data for _, tag in sentence]

    # 统计词性出现的次数
    counter = Counter(second_elements)
    counter_list = list(counter.items()) 
    counter_list_new = [ele for ele in counter_list if ele[0] not in punctuations]
    
    # 把子类别归类 ：例如 NN  NNS 为一大类都是名词
    # 定义词性标签集合: 名词  动词  形容词 副词 连词 限定词 代词
    tag_groups = {
        'noun': {'NN', 'NNS', 'NNP', 'NNPS'},
        'verb': {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
        'adjective': {'JJ', 'JJR', 'JJS'},
        'adverb': {'RB', 'RBR', 'RBS'},
        'conjunction': {'CC', 'IN'},
        'determiner': {'DT', 'PDT', 'WDT'},
        'pronoun': {'PRP', 'PRP$', 'WP', 'WP$'}
    }
    pos_counts = {key: 0 for key in tag_groups}

    # 进行类别求和操作并处理未分类词性
    unclassified_tags = [(tag, count) for tag, count in counter_list_new if not any(tag in group for group in tag_groups.values())]
    for tag, count in counter_list_new:
        for group, tags in tag_groups.items():
            if tag in tags:
                pos_counts[group] += count

    # 将未分类词性加入到最终结果列表中
    final_result = list(pos_counts.items()) + unclassified_tags

    
    # 按照出现次数逆序排序
    sorted_data = sorted(final_result, key=lambda x: x[1], reverse=True)

    return sorted_data

counter_tagging_type = static_tagging_type(tagged_sentence_list)

In [9]:
counter_tagging_type[:20]

[('noun', 14883),
 ('verb', 6176),
 ('determiner', 5768),
 ('adjective', 5549),
 ('conjunction', 4625),
 ('adverb', 893),
 ('TO', 775),
 ('MD', 385),
 ('CD', 253),
 ('pronoun', 247),
 ('POS', 93),
 ('WRB', 49),
 ('EX', 36),
 ('RP', 29),
 ('FW', 9),
 ('$', 6),
 ('SYM', 6),
 ('UH', 5)]

In [10]:
# comment_word = test_df[ele for ele in tagged_sentence_list]
# tagged_sentence_list
punctuations = {'.', ',', ';', ':', '(', ')', '[', ']', '{', '}', '!', '?', '``', "''"}

# comment_word = []
# for sentence in tagged_sentence_list:
#     temp = []
#     for word, _ in sentence:
#         if word not in punctuations:
#             temp.append(word)
#     comment_word.append(temp)

# 和上面两个for功能一样
comment_word = [[word for word, _ in sentence if word not in punctuations] for sentence in tagged_sentence_list]
comment_word[0]

['Creates', 'elastic', 'node', 'as', 'single', 'member', 'of', 'a', 'cluster']

In [11]:
# 计算每个子列表的长度
lengths = [len(sublist) for sublist in comment_word]
lengths[0]

9

In [12]:
# 计算平均长度
average_length = sum(lengths) / len(lengths)
average_length

10.140720081135903

In [13]:
# 超过平均长度的索引
indices_above_average = [index for index, length in enumerate(lengths) if length > average_length]
len(indices_above_average)

1461

In [14]:
# 长度为 10 和 11 的索引
indices_10and11 = [index for index, length in enumerate(lengths) if length ==10 or length == 11]
len(indices_10and11)

494

选择具有代表性的句子：长度接近平均长度的句子，或者选择长度超过平均长度的句子进行分析。

In [15]:
# 接近平均长度的句子
comment_sentence = [test_df.loc[i]['old_comment_raw'] for i in indices_10and11 ]
tagged_sentence_list_len10and11 = [tagged_sentence_list[i] for i in indices_10and11 ]
len(tagged_sentence_list_len10and11),len(comment_sentence)

(494, 494)

In [16]:
counter_tagging_type_len10and11 = static_tagging_type(tagged_sentence_list_len10and11)
counter_tagging_type_len10and11

[('noun', 1864),
 ('verb', 821),
 ('determiner', 800),
 ('conjunction', 652),
 ('adjective', 639),
 ('TO', 145),
 ('adverb', 95),
 ('MD', 37),
 ('pronoun', 29),
 ('CD', 21),
 ('WRB', 9),
 ('POS', 6),
 ('RP', 5),
 ('SYM', 3),
 ('EX', 2)]

In [21]:
alllenth_10and11 = sum(lengths[i] for i in indices_10and11) / len(indices_10and11)
alllenth_10and11

10.433198380566802

In [17]:
# 主要词性：名词，形容词，介词短语，代词，动词，副词，

main_type = ['NN','JJ','IN',   'PRP','VB',  'RB']

In [25]:
main_type = [('111',1),('222',2),('111',3)]
# 把子类别归类 ：例如 NN  NNS 为一大类都是名词
# 定义词性标签集合: 名词  动词  形容词 副词 连词 限定词 代词
tag_groups = {
        'noun': {'NN', 'NNS', 'NNP', 'NNPS'},
        'verb': {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
        'adjective': {'JJ', 'JJR', 'JJS'},
        'adverb': {'RB', 'RBR', 'RBS'},
        'conjunction': {'CC', 'IN'},
        'determiner': {'DT', 'PDT', 'WDT'},
        'pronoun': {'PRP', 'PRP$', 'WP', 'WP$'}
}
pos_counts = {key: 0 for key in tag_groups}

    # 进行类别求和操作并处理未分类词性
unclassified_tags = [(tag, count) for tag, count in main_type if not any(tag in group for group in tag_groups.values())]
for tag, count in main_type:
    for group, tags in tag_groups.items():
        if tag in tags:
            pos_counts[group] += count

    # 将未分类词性加入到最终结果列表中
final_result = list(pos_counts.items()) + unclassified_tags
final_result

[('noun', 0),
 ('verb', 0),
 ('adjective', 0),
 ('adverb', 0),
 ('conjunction', 0),
 ('determiner', 0),
 ('pronoun', 0),
 ('111', 1),
 ('222', 2),
 ('111', 3)]

In [26]:
# 定义词性标签集合
tag_groups = {
    'noun': {'NN', 'NNS', 'NNP', 'NNPS'},
    'verb': {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
    'adjective': {'JJ', 'JJR', 'JJS'},
    'adverb': {'RB', 'RBR', 'RBS'},
    'conjunction': {'CC', 'IN'},
    'determiner': {'DT', 'PDT', 'WDT'},
    'pronoun': {'PRP', 'PRP$', 'WP', 'WP$'}
}

# 原始列表
original_list = ['NN', 'NNS', 'VB', 'VBD', 'RB', 'RBR', 'EX']

# 创建一个映射字典
mapping = {tag: key for key, tags in tag_groups.items() for tag in tags}

# 将列表中的值替换为对应的键
mapped_list = [mapping.get(tag, tag) for tag in original_list]

print(mapped_list)


['noun', 'noun', 'verb', 'verb', 'adverb', 'adverb', 'EX']
